# 전기자극 파라미터 최적화 및 효과 분석

이 노트북은 신경재생을 위한 적응형 전기자극 시스템의 파라미터 최적화 및 효과 분석을 수행합니다. 다양한 전기자극 설정의 효과를 시뮬레이션하고 분석하여 신경 상태별 최적 자극 파라미터를 도출합니다.

## 1. 라이브러리 임포트

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import ParameterGrid
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
import joblib
import os
import sys
import time
import warnings
warnings.filterwarnings('ignore')

# 스타일 설정
plt.style.use('seaborn-whitegrid')
sns.set_theme(style="whitegrid")

# 경로 설정
sys.path.append('..')
from utils.stimulation_utils import simulate_stimulation_response, load_stimulation_data

# 랜덤 시드 설정
np.random.seed(42)

## 2. 전기자극 파라미터 정의 및 시뮬레이션 설정

In [ ]:
# 전기자극 파라미터 범위 정의
def define_parameter_space():
    """전기자극 파라미터 공간 정의"""
    param_space = {
        # 주파수 (Hz) - 신경 활성화에 직접 영향
        'frequency': [10, 20, 50, 100, 200],
        
        # 진폭 (mA) - 자극 강도 결정
        'amplitude': [0.5, 1.0, 2.0, 3.0, 5.0],
        
        # 펄스폭 (µs) - 총 에너지 전달량 영향
        'pulse_width': [100, 200, 300, 500, 1000],
        
        # 듀티 사이클 (%) - 자극 기간 대 휴식 기간의 비율
        'duty_cycle': [10, 25, 50, 75, 100],
        
        # 자극 기간 (분) - 총 자극 시간
        'duration': [15, 30, 45, 60, 90]
    }
    
    return param_space

# 전기자극의 신경재생 효과를 시뮬레이션하는 함수
def simulate_stimulation_effect(params, nerve_state='damaged', random_seed=None):
    """전기자극의 효과 시뮬레이션 (실제로는 복잡한 생물학적 모델을 사용할 것임)"""
    if random_seed is not None:
        np.random.seed(random_seed)
    
    # 각 신경 상태별 기준 효과 값 설정
    base_effectiveness = {
        'normal': 0.8,  # 정상 상태는 이미 효과가 높음
        'damaged': 0.2,  # 손상 상태는 기준 효과가 낮음
        'recovery': 0.5  # 회복 중인 상태는 중간 수준의 효과
    }
    
    # 파라미터 효과 계수 (실제로는 복잡한 비선형 관계가 있을 것임)
    # 신경 상태별로 효과적인 파라미터가 다름
    if nerve_state == 'normal':
        # 정상 상태에는 낮은 강도의 자극이 효과적
        freq_effect = -0.001 * (params['frequency'] - 50)**2 + 0.1  # 50Hz 근처가 최적
        amp_effect = -0.1 * params['amplitude'] + 0.2  # 낮은 진폭이 선호됨
        pw_effect = -0.0002 * (params['pulse_width'] - 200)**2 + 0.1  # 200µs 근처가 최적
        duty_effect = -0.0002 * (params['duty_cycle'] - 25)**2 + 0.1  # 낮은 듀티 사이클이 선호됨
        duration_effect = -0.0002 * (params['duration'] - 30)**2 + 0.05  # 중간 길이가 최적
        
    elif nerve_state == 'damaged':
        # 손상 상태에는 높은 강도의 자극이 효과적
        freq_effect = -0.0005 * (params['frequency'] - 100)**2 + 0.2  # 100Hz 근처가 최적
        amp_effect = 0.05 * params['amplitude'] + 0.1  # 높은 진폭이 선호됨
        pw_effect = -0.0001 * (params['pulse_width'] - 500)**2 + 0.15  # 500µs 근처가 최적
        duty_effect = -0.0001 * (params['duty_cycle'] - 75)**2 + 0.15  # 높은 듀티 사이클이 선호됨
        duration_effect = 0.001 * params['duration'] + 0.05  # 긴 시간이 더 효과적
        
    else:  # 'recovery'
        # 회복 상태에는 중간 강도의 자극이 효과적
        freq_effect = -0.0006 * (params['frequency'] - 75)**2 + 0.15  # 75Hz 근처가 최적
        amp_effect = -0.02 * (params['amplitude'] - 2.5)**2 + 0.15  # 중간 진폭이 선호됨
        pw_effect = -0.0001 * (params['pulse_width'] - 300)**2 + 0.12  # 300µs 근처가 최적
        duty_effect = -0.0001 * (params['duty_cycle'] - 50)**2 + 0.12  # 중간 듀티 사이클이 선호됨
        duration_effect = -0.0001 * (params['duration'] - 45)**2 + 0.1  # 중간 길이가 최적
    
    # 각 파라미터의 효과를 결합하여 총 효과 계산
    total_effect = base_effectiveness[nerve_state] + freq_effect + amp_effect + pw_effect + duty_effect + duration_effect
    
    # 신경재생 측정 지표들
    metrics = {
        # 축삭 성장률 (µm/일)
        'axon_growth_rate': max(0, min(100, total_effect * 70 + np.random.normal(0, 5))),
        
        # 신경성장인자 발현 수준 (상대적 단위)
        'growth_factor_expression': max(0, min(10, total_effect * 8 + np.random.normal(0, 0.5))),
        
        # 기능적 회복 점수 (0-100)
        'functional_recovery_score': max(0, min(100, total_effect * 80 + np.random.normal(0, 7))),
        
        # 신경 전도 속도 (m/s)
        'nerve_conduction_velocity': max(0, min(60, base_effectiveness[nerve_state] * 40 + total_effect * 15 + np.random.normal(0, 2))),
        
        # 부작용 심각도 (0-10, 낮을수록 좋음) - 자극이 너무 강하면 부작용 증가
        'side_effect_severity': max(0, min(10, (params['amplitude'] * params['pulse_width'] / 300) * 0.8 + 
                                    (params['frequency'] / 100) * 0.5 + (params['duty_cycle'] / 50) * 0.3 + 
                                    np.random.normal(0, 0.7)))
    }
    
    # 효과 점수 - 부작용을 고려한 종합 점수
    # 부작용이 심할수록 효과 점수 감소
    effectiveness_score = (0.3 * metrics['axon_growth_rate'] / 100 + 
                          0.2 * metrics['growth_factor_expression'] / 10 + 
                          0.3 * metrics['functional_recovery_score'] / 100 + 
                          0.2 * metrics['nerve_conduction_velocity'] / 60) * 
                          (1 - 0.1 * metrics['side_effect_severity'] / 10)
    
    # 0-100 스케일로 변환
    effectiveness_score = max(0, min(100, effectiveness_score * 100))
    metrics['effectiveness_score'] = effectiveness_score
    
    return metrics

# 파라미터 공간 생성
param_space = define_parameter_space()
print("전기자극 파라미터 공간:")
for param, values in param_space.items():
    print(f"{param}: {values}")

## 3. 그리드 탐색을 통한 초기 파라미터 평가

In [ ]:
# 그리드 탐색 함수 (모든 가능한 조합을 평가하기엔 너무 많으므로 샘플링)
def grid_search_sampling(param_space, nerve_states, n_samples=100):
    """파라미터 공간에서 무작위 샘플링하여 평가"""
    # 전체 파라미터 그리드 생성
    param_grid = list(ParameterGrid(param_space))
    total_combinations = len(param_grid)
    print(f"총 파라미터 조합 수: {total_combinations}")
    
    # 샘플 수 조정 (너무 많으면 시간이 오래 걸림)
    n_samples = min(n_samples, total_combinations)
    
    # 무작위 샘플링
    sampled_indices = np.random.choice(total_combinations, size=n_samples, replace=False)
    sampled_params = [param_grid[i] for i in sampled_indices]
    
    # 결과 저장 데이터프레임 준비
    results = []
    
    # 각 신경 상태에 대해 평가
    for state in nerve_states:
        print(f"\n신경 상태 '{state}'에 대한 파라미터 평가 중...")
        start_time = time.time()
        
        for i, params in enumerate(sampled_params):
            if i % 10 == 0:
                print(f"  진행률: {i}/{n_samples} ({i/n_samples*100:.1f}%)")
                
            # 효과 시뮬레이션
            metrics = simulate_stimulation_effect(params, nerve_state=state, random_seed=i)
            
            # 결과 저장
            row = {
                'nerve_state': state,
                **params,
                **metrics
            }
            results.append(row)
        
        elapsed_time = time.time() - start_time
        print(f"  완료! 소요 시간: {elapsed_time:.2f}초")
    
    # 데이터프레임으로 변환
    results_df = pd.DataFrame(results)
    
    return results_df

# 신경 상태 정의
nerve_states = ['normal', 'damaged', 'recovery']

# 그리드 탐색 실행
results_df = grid_search_sampling(param_space, nerve_states, n_samples=150)

## 4. 결과 분석 및 시각화

In [ ]:
# 결과 요약
print("그리드 탐색 결과 요약:")
print(f"총 평가된 파라미터 조합: {len(results_df)}")

# 각 신경 상태별 최적 파라미터 조합 찾기
best_params = {}
for state in nerve_states:
    state_df = results_df[results_df['nerve_state'] == state]
    best_idx = state_df['effectiveness_score'].idxmax()
    best_row = results_df.loc[best_idx]
    
    best_params[state] = {
        'frequency': best_row['frequency'],
        'amplitude': best_row['amplitude'],
        'pulse_width': best_row['pulse_width'],
        'duty_cycle': best_row['duty_cycle'],
        'duration': best_row['duration'],
        'effectiveness_score': best_row['effectiveness_score'],
        'axon_growth_rate': best_row['axon_growth_rate'],
        'side_effect_severity': best_row['side_effect_severity']
    }
    
    print(f"\n신경 상태 '{state}'에 대한 최적 파라미터:")
    for param, value in best_params[state].items():
        print(f"  {param}: {value:.2f}" if isinstance(value, float) else f"  {param}: {value}")

# 효과 점수 분포 시각화
plt.figure(figsize=(10, 6))
sns.boxplot(x='nerve_state', y='effectiveness_score', data=results_df, palette=['green', 'red', 'blue'])
plt.title('신경 상태별 효과 점수 분포')
plt.xlabel('신경 상태')
plt.ylabel('효과 점수 (0-100)')
plt.grid(True, alpha=0.3)
plt.show()

# 파라미터별 효과 관계 시각화
params_to_plot = list(param_space.keys())
metrics_to_plot = ['effectiveness_score', 'axon_growth_rate', 'side_effect_severity']

for state in nerve_states:
    state_df = results_df[results_df['nerve_state'] == state]
    
    for param in params_to_plot:
        plt.figure(figsize=(12, 4))
        plt.suptitle(f"신경 상태: {state} - {param}의 영향", fontsize=14)
        
        for i, metric in enumerate(metrics_to_plot):
            plt.subplot(1, 3, i+1)
            sns.scatterplot(x=param, y=metric, data=state_df, alpha=0.6)
            
            # 추세선 추가
            try:
                sns.regplot(x=param, y=metric, data=state_df, scatter=False, ci=None, color='red')
            except:
                pass
            
            plt.title(f"{metric}")
            plt.grid(True, alpha=0.3)
            
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.show()

## 5. 가우시안 프로세스를 통한 파라미터 최적화

In [ ]:
# 가우시안 프로세스를 이용한 파라미터 최적화
def optimize_parameters_with_gp(results_df, nerve_state, param_ranges):
    """가우시안 프로세스 회귀를 사용한 파라미터 최적화"""
    # 해당 신경 상태의 데이터만 선택
    state_df = results_df[results_df['nerve_state'] == nerve_state].copy()
    
    # 입력 특성(파라미터)과 타겟(효과 점수) 준비
    param_cols = list(param_ranges.keys())
    X = state_df[param_cols].values
    y = state_df['effectiveness_score'].values
    
    # 특성 정규화
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # 가우시안 프로세스 모델 정의
    kernel = C(1.0, (1e-3, 1e3)) * RBF([1.0] * len(param_cols), (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, random_state=42)
    
    # 모델 학습
    gp.fit(X_scaled, y)
    
    # 최적화를 위한 목적 함수 (음수 효과 점수 - 최소화 문제로 변환)
    def objective(x_scaled):
        x_scaled = x_scaled.reshape(1, -1)
        # 예측값의 음수 반환 (최소화 문제)
        return -gp.predict(x_scaled)[0]
    
    # 여러 시작점에서 최적화 시도
    best_score = float('-inf')
    best_params_scaled = None
    num_starts = 10
    
    for i in range(num_starts):
        # 무작위 시작점 (정규화된 공간)
        x0 = np.random.randn(len(param_cols))
        
        # 최적화 (L-BFGS-B 알고리즘 사용)
        bounds = [(-3, 3)] * len(param_cols)  # 정규화된 공간에서의 경계 (-3σ ~ +3σ)
        res = minimize(objective, x0, method='L-BFGS-B', bounds=bounds)
        
        # 최적 점수 업데이트
        if -res.fun > best_score:
            best_score = -res.fun
            best_params_scaled = res.x
    
    # 최적 파라미터를 원래 스케일로 변환
    best_params_orig = scaler.inverse_transform(best_params_scaled.reshape(1, -1))[0]
    
    # 파라미터 범위에 맞게 조정
    optimized_params = {}
    for i, param in enumerate(param_cols):
        # 연속적인 최적값을 가장 가까운 허용된 값으로 매핑
        allowed_values = param_ranges[param]
        closest_idx = np.argmin(np.abs(np.array(allowed_values) - best_params_orig[i]))
        optimized_params[param] = allowed_values[closest_idx]
    
    # 최적화된 파라미터로 다시 평가
    final_metrics = simulate_stimulation_effect(optimized_params, nerve_state=nerve_state, random_seed=42)
    
    return {
        'optimized_params': optimized_params,
        'predicted_score': best_score,
        'actual_score': final_metrics['effectiveness_score'],
        'metrics': final_metrics,
        'gp_model': gp,
        'scaler': scaler
    }

# 각 신경 상태에 대해 최적화 수행
optimized_results = {}
for state in nerve_states:
    print(f"\n신경 상태 '{state}'에 대한 가우시안 프로세스 최적화 중...")
    optimized_results[state] = optimize_parameters_with_gp(results_df, state, param_space)
    
    print(f"  최적화된 파라미터:")
    for param, value in optimized_results[state]['optimized_params'].items():
        print(f"    {param}: {value}")
    print(f"  예측된 효과 점수: {optimized_results[state]['predicted_score']:.2f}")
    print(f"  실제 효과 점수: {optimized_results[state]['actual_score']:.2f}")
    print(f"  부작용 심각도: {optimized_results[state]['metrics']['side_effect_severity']:.2f}")

## 6. 최적화 결과 시각화 및 분석

In [ ]:
# 원래 그리드 탐색 결과와 최적화 결과 비교
comparison_data = []

for state in nerve_states:
    # 그리드 탐색 결과의 최고 점수
    grid_best_score = best_params[state]['effectiveness_score']
    
    # 최적화 결과 점수
    opt_score = optimized_results[state]['actual_score']
    
    comparison_data.append({
        'nerve_state': state,
        'method': 'Grid Search',
        'effectiveness_score': grid_best_score
    })
    comparison_data.append({
        'nerve_state': state,
        'method': 'GP Optimization',
        'effectiveness_score': opt_score
    })

comparison_df = pd.DataFrame(comparison_data)

# 비교 시각화
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='nerve_state', y='effectiveness_score', hue='method', data=comparison_df)
plt.title('그리드 탐색 vs 가우시안 프로세스 최적화 비교')
plt.xlabel('신경 상태')
plt.ylabel('효과 점수 (0-100)')
plt.grid(True, alpha=0.3)

# 값 표시
for i, p in enumerate(ax.patches):
    ax.annotate(f'{p.get_height():.1f}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', xytext = (0, 5), 
                textcoords = 'offset points')

plt.tight_layout()
plt.show()

In [ ]:
# 최적화된 파라미터 설정 시각화
def plot_optimized_parameters(optimized_results, param_space):
    """최적화된 파라미터 설정 시각화"""
    # 파라미터별 최적값 추출
    params = list(param_space.keys())
    states = list(optimized_results.keys())
    
    # 각 파라미터별 시각화
    fig, axes = plt.subplots(len(params), 1, figsize=(10, 3*len(params)))
    
    for i, param in enumerate(params):
        ax = axes[i]
        
        # 허용 가능한 값들 범위
        param_values = param_space[param]
        ax.set_xlim(min(param_values) - (max(param_values) - min(param_values))*0.1, 
                   max(param_values) + (max(param_values) - min(param_values))*0.1)
        
        # 상태별 색상 정의
        colors = {'normal': 'green', 'damaged': 'red', 'recovery': 'blue'}
        
        # 각 신경 상태별 최적값 표시
        for state in states:
            value = optimized_results[state]['optimized_params'][param]
            ax.plot([value], [0], 'o', markersize=10, color=colors[state], label=state if i == 0 else '')
            ax.annotate(f"{value}", (value, 0), xytext=(0, 10), 
                       textcoords='offset points', ha='center', va='bottom',
                       color=colors[state], fontweight='bold')
        
        # 허용 가능한 값들 표시
        ax.scatter(param_values, [0] * len(param_values), marker='|', s=100, color='gray', alpha=0.5)
        
        # 축 설정
        ax.set_yticks([])
        ax.set_title(f'{param}', fontsize=12)
        ax.grid(True, alpha=0.3)
        
        # 단위 표시
        units = {
            'frequency': 'Hz',
            'amplitude': 'mA',
            'pulse_width': 'µs',
            'duty_cycle': '%',
            'duration': 'min'
        }
        if param in units:
            ax.set_xlabel(f"[{units[param]}]")
    
    # 범례는 첫 번째 플롯에만 추가
    if len(params) > 0:
        axes[0].legend(title='신경 상태', loc='upper right')
    
    plt.suptitle('신경 상태별 최적화된 전기자극 파라미터', fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

# 최적화된 파라미터 시각화
plot_optimized_parameters(optimized_results, param_space)

## 7. 최적화된 파라미터의 성능 평가

In [ ]:
# 최적화된 파라미터의 종합 성능 평가
def evaluate_optimized_parameters(optimized_results, nerve_states, n_trials=30):
    """최적화된 파라미터의 여러 시도에 걸친 성능 평가"""
    performance_data = []
    
    for state in nerve_states:
        params = optimized_results[state]['optimized_params']
        
        # 여러 번의 시도를 통한 성능 평가
        print(f"신경 상태 '{state}'에 대한 평가 중...")
        
        for trial in range(n_trials):
            # 매번 다른 랜덤 시드로 시뮬레이션
            metrics = simulate_stimulation_effect(params, nerve_state=state, random_seed=trial+100)
            
            # 결과 저장
            performance_data.append({
                'nerve_state': state,
                'trial': trial,
                **params,
                **metrics
            })
    
    performance_df = pd.DataFrame(performance_data)
    return performance_df

# 성능 평가 실행
performance_df = evaluate_optimized_parameters(optimized_results, nerve_states)

# 성능 통계 요약
print("최적화된 파라미터의 성능 통계:")
performance_summary = performance_df.groupby('nerve_state')[
    'effectiveness_score', 'axon_growth_rate', 'growth_factor_expression', 
    'functional_recovery_score', 'nerve_conduction_velocity', 'side_effect_severity'
].agg(['mean', 'std', 'min', 'max'])

print(performance_summary)

# 성능 분포 시각화
plt.figure(figsize=(12, 8))

metrics_to_plot = [
    'effectiveness_score', 'axon_growth_rate', 'growth_factor_expression', 
    'functional_recovery_score', 'nerve_conduction_velocity', 'side_effect_severity'
]

for i, metric in enumerate(metrics_to_plot):
    plt.subplot(2, 3, i+1)
    sns.violinplot(x='nerve_state', y=metric, data=performance_df, palette=['green', 'red', 'blue'])
    plt.title(metric)
    plt.grid(True, alpha=0.3)
    
    # Y축 레이블 간소화
    if i % 3 != 0:  # 각 행의 첫 번째 플롯이 아니면
        plt.ylabel('')
    
    # X축 레이블 간소화
    if i < 3:  # 마지막 행이 아니면
        plt.xlabel('')

plt.suptitle('최적화된 파라미터의 성능 분포', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

## 8. 신경 상태에 따른 적응형 전기자극 프로토콜 정의

In [ ]:
# 적응형 전기자극 프로토콜 정의
def define_adaptive_stimulation_protocol(optimized_results):
    """신경 상태별 최적 파라미터를 기반으로 적응형 프로토콜 정의"""
    protocol = {}
    
    for state, result in optimized_results.items():
        params = result['optimized_params']
        metrics = result['metrics']
        
        # 주요 설정 정리
        protocol[state] = {
            'parameters': {
                'frequency': params['frequency'],
                'amplitude': params['amplitude'],
                'pulse_width': params['pulse_width'],
                'duty_cycle': params['duty_cycle'],
                'duration': params['duration']
            },
            'expected_metrics': {
                'effectiveness_score': metrics['effectiveness_score'],
                'axon_growth_rate': metrics['axon_growth_rate'],
                'growth_factor_expression': metrics['growth_factor_expression'],
                'functional_recovery_score': metrics['functional_recovery_score'],
                'nerve_conduction_velocity': metrics['nerve_conduction_velocity'],
                'side_effect_severity': metrics['side_effect_severity']
            },
            'protocol_name': f"{state.capitalize()} State ES Protocol",
            'recommendations': get_protocol_recommendations(state, params, metrics)
        }
    
    return protocol

# 상태별 추천 사항 정의 함수
def get_protocol_recommendations(state, params, metrics):
    """각 신경 상태에 맞는 권장 사항 생성"""
    if state == 'normal':
        return [
            "1일 1회 적용 권장",
            "자극 중 환자 상태 모니터링 필요",
            "부작용 발생 시 진폭 10% 감소 고려",
            "2주 후 효과 평가 및 파라미터 재조정 권장",
            "생체역학적 재활 훈련과 병행 시 효과 상승"
        ]
    elif state == 'damaged':
        return [
            "초기 1주일은 1일 2회 적용 권장",
            "상태 개선에 따라 점진적으로 자극 강도 조절",
            "통증 관리와 병행",
            "자극 후 24시간 이내 염증 마커 모니터링",
            "손상 정도에 따라 진폭 개별화 (중증 손상 시 +10%)",
            "1주일 간격으로 BDNF/GDNF 발현 수준 평가"
        ]
    else:  # 'recovery'
        return [
            "1일 1회 적용 권장",
            "점진적인 기능적 훈련과 병행",
            "2주 간격으로 신경 전도 속도 평가",
            "재생 진행에 따라 주파수 점진적 감소 고려",
            "감각/운동 기능 회복에 따른 파라미터 미세 조정",
            "완전 회복 시 유지 요법으로 변경 (주 1-2회)"
        ]

# 적응형 프로토콜 정의
adaptive_protocol = define_adaptive_stimulation_protocol(optimized_results)

# 프로토콜 출력
for state, protocol in adaptive_protocol.items():
    print(f"\n*** {protocol['protocol_name']} ***")
    
    print("\n파라미터 설정:")
    for param, value in protocol['parameters'].items():
        # 단위 추가
        units = {
            'frequency': 'Hz',
            'amplitude': 'mA',
            'pulse_width': 'µs',
            'duty_cycle': '%',
            'duration': 'min'
        }
        unit = units.get(param, '')
        print(f"  {param}: {value} {unit}")
    
    print("\n예상 효과:")
    for metric, value in protocol['expected_metrics'].items():
        print(f"  {metric}: {value:.2f}")
    
    print("\n권장 사항:")
    for i, rec in enumerate(protocol['recommendations']):
        print(f"  {i+1}. {rec}")
    
    print("\n" + "-"*50)

## 9. 모델 저장 및 내보내기

In [ ]:
# 최적화 결과 및 프로토콜 저장
def save_optimization_results(optimized_results, adaptive_protocol):
    """최적화 결과 및 프로토콜 저장"""
    # 저장 디렉토리 생성
    output_dir = "../models/stimulation_protocols"
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. 프로토콜 JSON 저장
    import json
    protocol_path = os.path.join(output_dir, "adaptive_protocols.json")
    
    # JSON 직렬화 가능하도록 NumPy 타입 변환
    def convert_numpy_types(obj):
        if isinstance(obj, dict):
            return {k: convert_numpy_types(v) for k, v in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy_types(item) for item in obj]
        elif isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj
    
    # 프로토콜 저장
    with open(protocol_path, 'w') as f:
        json.dump(convert_numpy_types(adaptive_protocol), f, indent=4)
    print(f"적응형 프로토콜이 {protocol_path}에 저장되었습니다.")
    
    # 2. GP 모델 저장
    for state, result in optimized_results.items():
        # GP 모델 저장
        model_path = os.path.join(output_dir, f"gp_model_{state}.pkl")
        joblib.dump(result['gp_model'], model_path)
        
        # 스케일러 저장
        scaler_path = os.path.join(output_dir, f"scaler_{state}.pkl")
        joblib.dump(result['scaler'], scaler_path)
    
    print(f"GP 모델 및 스케일러가 {output_dir}에 저장되었습니다.")
    
    # 3. 성능 평가 결과 저장
    performance_path = os.path.join(output_dir, "performance_evaluation.csv")
    performance_df.to_csv(performance_path, index=False)
    print(f"성능 평가 결과가 {performance_path}에 저장되었습니다.")

# 결과 저장
save_optimization_results(optimized_results, adaptive_protocol)

## 10. 결론 및 다음 단계

본 노트북은 신경재생을 위한 적응형 전기자극 시스템의 파라미터 최적화 및 효과 분석을 수행했습니다. 주요 내용은 다음과 같습니다:

1. **파라미터 공간 설정**: 주파수, 진폭, 펄스폭, 듀티 사이클, 자극 기간 등 다양한 전기자극 파라미터를 정의했습니다.
2. **효과 시뮬레이션**: 다양한 신경 상태(정상, 손상, 회복)에 따른 전기자극의 효과를 시뮬레이션했습니다.
3. **그리드 탐색**: 파라미터 공간을 샘플링하여 초기 평가를 진행했습니다.
4. **가우시안 프로세스 최적화**: 그리드 탐색 결과를 바탕으로 가우시안 프로세스 회귀를 통해 파라미터를 더 정밀하게 최적화했습니다.
5. **성능 평가**: 최적화된 파라미터의 성능을 여러 시도에 걸쳐 평가했습니다.
6. **적응형 프로토콜 정의**: 각 신경 상태별로 최적화된 파라미터를 기반으로 적응형 전기자극 프로토콜을 정의했습니다.

### 주요 발견:

- **상태별 최적 파라미터 차이**: 정상, 손상, 회복 상태에 따라 최적의 전기자극 파라미터가 크게 다르다는 것을 확인했습니다.
- **부작용과 효과 사이의 균형**: 자극 강도를 높일수록 효과가 커지지만, 동시에 부작용도 증가하는 경향이 있습니다.
- **개인화의 중요성**: 신경 상태별 최적화 결과의 변동성을 고려할 때, 개인별 미세 조정이 중요함을 알 수 있습니다.

### 다음 단계:

1. **실제 데이터 기반 모델 개선**: 시뮬레이션 대신 실제 신경재생 데이터를 사용하여 모델을 개선합니다.
2. **실시간 적응 알고리즘 개발**: 신경 상태 변화에 따라 자동으로 파라미터를 조정하는 알고리즘을 개발합니다.
3. **개인별 파라미터 최적화**: 개인의 특성과 신경 손상 패턴에 맞춘 파라미터 최적화 방법을 연구합니다.
4. **임상 검증**: 정의된 프로토콜의 효과와 안전성을 임상 시험을 통해 검증합니다.
5. **모바일 애플리케이션 개발**: 정의된 프로토콜을 적용하고 모니터링할 수 있는 모바일 애플리케이션을 개발합니다.

이러한 결과는 신경재생을 위한 맞춤형 전기자극 시스템 개발에 중요한 기초를 제공할 것입니다.